In [ ]:
import polars as pl

stat_cols = pl.selectors.ends_with('_ks', '_kw', '_med')

df = (
    pl.read_csv("C:/Users/ben.hardcastle/Downloads/fit_non_paramertic_trial_all.csv")
    .select('unit_id', stat_cols, pl.lit('trial').alias('interval'))
    # ----------------------------------------------------------------- #
    .vstack(
        pl.read_csv("C:/Users/ben.hardcastle/Downloads/fit_non_paramertic_quiescent_all.csv")
        .select('unit_id', stat_cols, pl.lit('baseline').alias('interval'))
    )
    .vstack(
        pl.read_csv("C:/Users/ben.hardcastle/Downloads/fit_non_paramertic_response_all.csv")
        .select('unit_id', stat_cols, pl.lit('response').alias('interval'))
    )
    .unpivot(
        index=['unit_id' ,'interval'],
        variable_name='label',
    )
    .with_columns(
        pl.col('label').str.split('_').list.get(0).alias('result_type'),
        pl.col('label').str.split('_').list.get(1).str.strip_prefix('block').cast(pl.Int8).alias('block_index'),
        pl.col('label').str.split('_').list.get(-1).alias('test')
    )
    .drop('label')
    .pivot(
        on='result_type',
        values='value',
    )
    .group_by('test', 'unit_id', 'interval')
    .agg(
        pl.col('p').min().alias('p_min'),
        pl.col('D').max().alias('D_max'),
        pl.col('F').max().alias('F_max'),
    )
    .pivot(
        on=['interval'],
        values=['p_min', 'D_max', 'F_max'],
    )
)
def get_rename_map(test: str, drop: str) -> dict:
    return {
        f'{stat}_{interval}': f'{test}_{stat}_{interval}'
        for interval in ['trial', 'baseline', 'response']
        for stat in [s for s in ('p_min', 'D_max', 'F_max') if not s.startswith(drop)]
    }

for test, drop in [('ks', 'F_'), ('kw', 'D_'), ('med', 'F_')]:
    df.filter(pl.col('test') == test).drop('test', pl.selectors.starts_with(drop)).rename(get_rename_map(test, drop)).write_parquet(f"//allen/programs/mindscope/workgroups/dynamicrouting/ben/{test}_test.parquet")
df

wrote parquet file
wrote parquet file
wrote parquet file


test,unit_id,p_min_trial,p_min_response,p_min_baseline,D_max_trial,D_max_response,D_max_baseline,F_max_trial,F_max_response,F_max_baseline
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""med""","""714748_2024-06-25_E-192""",0.000008,0.000674,0.002604,20.009333,11.558601,9.066036,null,null,null
"""med""","""664851_2023-11-16_E-188""",8.5460e-19,8.1823e-12,7.8279e-17,78.369534,46.72158,69.452441,null,null,null
"""ks""","""664851_2023-11-14_C-10""",1.0,1.0,1.0,0.017442,0.022989,0.011765,null,null,null
"""med""","""737403_2024-09-27_E-616""",5.6741e-27,3.5499e-24,2.3057e-23,115.648689,102.886051,99.179963,null,null,null
"""med""","""715710_2024-07-17_B-175""",2.1307e-7,0.000002,0.016167,26.910785,22.813051,5.784649,null,null,null
…,…,…,…,…,…,…,…,…,…,…
"""ks""","""703882_2024-04-25_A-562""",0.831458,0.956195,1.0,0.078196,0.063522,0.026337,null,null,null
"""ks""","""649943_2023-02-13_F-449""",0.000007,0.000012,2.1800e-8,0.324408,0.317174,0.38764,null,null,null
"""kw""","""626791_2022-08-17_F-226""",1.6272e-35,2.1299e-34,2.9684e-30,null,null,null,154.701489,149.590988,130.639973


In [ ]:
get_rename_map('ks')

{'p_min_trial': 'ks_p_min_trial',
 'D_max_trial': 'ks_D_max_trial',
 'F_max_trial': 'ks_F_max_trial',
 'p_min_baseline': 'ks_p_min_baseline',
 'D_max_baseline': 'ks_D_max_baseline',
 'F_max_baseline': 'ks_F_max_baseline',
 'p_min_response': 'ks_p_min_response',
 'D_max_response': 'ks_D_max_response',
 'F_max_response': 'ks_F_max_response'}

In [ ]:
pl.read_parquet(
    "//allen/programs/mindscope/workgroups/dynamicrouting/ben/ks_test.parquet"
)

unit_id,ks_p_min_trial,ks_p_min_response,ks_p_min_baseline,ks_D_max_trial,ks_D_max_response,ks_D_max_baseline
str,f64,f64,f64,f64,f64,f64
"""664851_2023-11-14_C-10""",1.0,1.0,1.0,0.017442,0.022989,0.011765
"""626791_2022-08-15_F-210""",1.8347e-12,3.1094e-10,0.004286,0.475583,0.429739,0.244722
"""668755_2023-08-31_F-44""",2.5926e-25,2.9667e-19,7.9815e-18,0.662921,0.578836,0.557721
"""733780_2024-09-04_E-282""",0.993409,0.983241,1.0,0.050924,0.055517,0.030295
"""649943_2023-02-14_D-692""",3.5735e-10,3.3295e-8,3.5735e-10,0.423077,0.379121,0.423077
…,…,…,…,…,…,…
"""674562_2023-10-04_B-255""",6.2704e-10,0.000002,2.1859e-10,0.558187,0.450731,0.570906
"""708016_2024-05-01_F-142""",2.2063e-30,8.5076e-23,9.9488e-12,0.718616,0.630004,0.468398
"""703882_2024-04-25_A-562""",0.831458,0.956195,1.0,0.078196,0.063522,0.026337


: 